In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [5]:
from expiries import dict_expiries

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
bnf_1min = pd.read_csv("../data/nifty_min.csv")

In [8]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.tail()

,index,datetime,o,h,l,c,v
750760,nifty,2025-03-28 15:25:00,23500.2500,23502.5000,23485.7500,23490.4000,0
750761,nifty,2025-03-28 15:26:00,23490.7500,23494.3500,23486.7500,23494.0500,0
750762,nifty,2025-03-28 15:27:00,23494.2000,23497.4500,23489.7500,23496.8000,0
750763,nifty,2025-03-28 15:28:00,23497.1000,23500.4500,23491.0500,23492.0000,0
750764,nifty,2025-03-28 15:29:00,23492.2500,23503.1000,23450.2000,23495.1500,0


In [9]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [10]:
dict_expiries

{'nifty': [datetime.datetime(2017, 1, 25, 0, 0),
  datetime.datetime(2017, 2, 23, 0, 0),
  datetime.datetime(2017, 3, 30, 0, 0),
  datetime.datetime(2017, 4, 27, 0, 0),
  datetime.datetime(2017, 5, 25, 0, 0),
  datetime.datetime(2017, 6, 29, 0, 0),
  datetime.datetime(2017, 7, 27, 0, 0),
  datetime.datetime(2017, 8, 31, 0, 0),
  datetime.datetime(2017, 9, 28, 0, 0),
  datetime.datetime(2017, 10, 26, 0, 0),
  datetime.datetime(2017, 11, 30, 0, 0),
  datetime.datetime(2017, 12, 28, 0, 0),
  datetime.datetime(2018, 1, 25, 0, 0),
  datetime.datetime(2018, 2, 22, 0, 0),
  datetime.datetime(2018, 3, 28, 0, 0),
  datetime.datetime(2018, 4, 26, 0, 0),
  datetime.datetime(2018, 5, 31, 0, 0),
  datetime.datetime(2018, 6, 28, 0, 0),
  datetime.datetime(2018, 7, 26, 0, 0),
  datetime.datetime(2018, 8, 30, 0, 0),
  datetime.datetime(2018, 9, 27, 0, 0),
  datetime.datetime(2018, 10, 25, 0, 0),
  datetime.datetime(2018, 11, 29, 0, 0),
  datetime.datetime(2018, 12, 27, 0, 0),
  datetime.datetime(2019,

In [11]:
from datetime import date
from bisect import bisect_right

def get_next_expiry(input_date, index_symbol):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_right(expiry_dates, input_date)
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [12]:
index_ = 'nifty'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8
    PORTFOLIO_ = 1_00_00_000

In [13]:
import pandas as pd

async def add_atr(df, period=14):
    """
    Adds an 'ATR' column to the DataFrame using Wilder's smoothing (like TradingView).
    
    Parameters:
    df (pd.DataFrame): Must contain 'h', 'l', 'c' columns for high, low, close
    period (int): ATR period (default 14)
    
    Returns:
    pd.DataFrame: With 'ATR' column added
    """
    high = df['h']
    low = df['l']
    close = df['c']

    # True Range
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    # ATR with Wilder's smoothing (like an EMA with alpha=1/period)
    atr = tr.ewm(alpha=1/period, adjust=False).mean()

    df['ATR'] = atr
    return df


In [76]:
async def execute(df, multiplier, open_range_till, tf, offset):

    start_date = dt.date(2019, 1, 1)
    end_date = dt.date(2025, 3, 31)

    # start_date = dt.date(2019, 3, 1)
    # end_date = dt.date(2025, 3, 31)
    
    current_date = start_date

    # Convert datetime column to date for comparison
    df_dates = df['datetime'].dt.date
    
    tradebook = []

    cut_off_time = dt.time(15, 0)

    while(current_date < end_date):
        
        if current_date in df_dates.values:
            print(current_date)
            today_df = df[df['datetime'].dt.date == current_date]
            today_df.reset_index(drop=True, inplace=True)

            open_range_df = today_df[today_df['datetime'].dt.time < open_range_till]

            morning_open = today_df['o'].iloc[0]
            morning_close = today_df['c'].iloc[0]
            morning_atr = today_df['ATR'].iloc[0]

            high_level = morning_close + (multiplier * morning_atr)
            low_level = morning_close - (multiplier * morning_atr)

            high_strike = int(round(high_level / STRIKE_SPREAD_) * STRIKE_SPREAD_)
            low_strike = int(round(low_level / STRIKE_SPREAD_) * STRIKE_SPREAD_)

            expiry = get_next_expiry(current_date, index_)

            # print(morning_close, morning_atr, high_level, low_level)

            # Initialize flags
            high_flag_1 = False
            high_flag_2 = False
            high_flag_3 = False
            low_flag_1 = False
            low_flag_2 = False
            low_flag_3 = False
            
            data_fetched_ce = False
            data_fetched_pe = False
            in_trade_long = False
            in_trade_short = False

            trade_num = 0
            max_trades = 2

            for i, row in today_df.iterrows():
                close_price = row['c']
                timestamp = row['datetime']
                # print(timestamp, timestamp.time(), type(timestamp))
                # HIGH breakout sequence
                if not high_flag_1 and close_price >= high_level:
                    # print(f'First High Breach @ {timestamp}')
                    high_flag_1 = True

                elif high_flag_1 and not high_flag_2 and close_price <= high_level:
                    # print(f'Retracement done for High @ {timestamp}')
                    high_flag_2 = True

                if high_flag_2:
                    # Entry Triggered
                    if not data_fetched_ce:
                        ce_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=low_strike,
                            asset_class='C',
                            start_date=current_date,
                            start_time=dt.time(9, 15),
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        data_fetched_ce = True

                    if not in_trade_short and trade_num < max_trades and timestamp.time() <= cut_off_time:
                        if not isinstance(ce_df, str) and ce_df is not None:
                            if (len(ce_df) * int(tf[:-1])) > (200 / int(tf[:-1])):
                                ce_df = resample(ce_df, tf, offset)
                                ce_df_pandas = ce_df.to_pandas()
                                # print(ce_df_pandas.to_string())
                                subset_df = ce_df_pandas[ce_df_pandas['datetime'] >= timestamp]
                                ce_entry_price = subset_df.iloc[0]['c']
                            else:
                                ce_entry_price = float('nan')    
                        else:
                            ce_entry_price = float('nan')
    
                        trade = {
                            'date': current_date,
                            'high level': high_level,
                            'low level': low_level,
                            'atr_multiplier': multiplier,
                            'side': 'SHORT',
                            'strike': low_strike,
                            'type': 'CE',
                            'expiry': expiry,
                            'entry_time': timestamp,
                            'entry price': ce_entry_price,
                        }
                        # print(trade)
                        tradebook.append(trade)
                        in_trade_short = True
                        trade_num += 1
    
                    if in_trade_short:
                        if close_price >= high_level:
                            # SL Hit
                            # print('SL Condition CE')
                            for trade in tradebook:
                                if (
                                    trade['type'] == 'CE' and 
                                    trade['date'] == current_date and 
                                    'exit_time' not in trade
                                ):
                                    df_row = ce_df_pandas[ce_df_pandas['datetime'] <= timestamp]
                                    # print(df_row.to_string())
                                    ce_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                                    trade['exit price'] =ce_exit_price
                                    trade['exit_time'] = df_row['datetime'].iloc[-1] if len(df_row) != 0 else float('nan')
                                    trade['remarks'] = 'SL Hit'
                                    
                            in_trade_short = False
                            high_flag_2 = False
                            
                        elif timestamp.time() >= dt.time(15, 20):
                            #EOD Exit
                            # print('EOD Exit Condition CE')
                            for trade in tradebook:
                                if (
                                    trade['type'] == 'CE' and 
                                    trade['date'] == current_date and 
                                    'exit_time' not in trade
                                ):
                                    df_row = ce_df_pandas[ce_df_pandas['datetime'] <= timestamp]
                                    # print("EOD CE Side")
                                    # print(df_row.to_string())
                                    ce_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                                    trade['exit price'] =ce_exit_price
                                    trade['exit_time'] = df_row['datetime'].iloc[-1] if len(df_row) != 0 else float('nan')
                                    trade['remarks'] = 'EOD Exit'
                                    
                            in_trade_short = False
                            high_flag_2 = False

                # LOW breakdown sequence
                if not low_flag_1 and close_price <= low_level:
                    # print(f'First Low Breach @ {timestamp}')
                    low_flag_1 = True

                elif low_flag_1 and not low_flag_2 and close_price >= low_level:
                    # print(f'Retracement done for Low @ {timestamp}')
                    low_flag_2 = True

                if low_flag_2:
                    # Entry Triggered
                    if not data_fetched_pe:
                        pe_df = await fetch_data(
                            index=index_,
                            expiry=expiry,
                            strike=high_strike,
                            asset_class='P',
                            start_date=current_date,
                            start_time=dt.time(9, 15),
                            end_date=current_date,
                            end_time=dt.time(15, 30),
                        )
                        data_fetched_pe = True

                    if not in_trade_long and trade_num < max_trades and timestamp.time() <= cut_off_time:
                        if not isinstance(pe_df, str) and pe_df is not None:
                            if (len(pe_df) * int(tf[:-1])) > (200 / int(tf[:-1])):
                                pe_df = resample(pe_df, tf, offset)
                                pe_df_pandas = pe_df.to_pandas()
                                # print(pe_df_pandas.to_string())
                                subset_df = pe_df_pandas[pe_df_pandas['datetime'] >= timestamp]
                                pe_entry_price = subset_df.iloc[0]['c']
                            else:
                                pe_entry_price = float('nan')    
                        else:
                            pe_entry_price = float('nan')
    
                        trade = {
                            'date': current_date,
                            'high level': high_level,
                            'low level': low_level,
                            'atr_multiplier': multiplier,
                            'side': 'LONG',
                            'strike': high_strike,
                            'type': 'PE',
                            'expiry': expiry,
                            'entry_time': timestamp,
                            'entry price': pe_entry_price,
                        }
                        # print(trade)
                        tradebook.append(trade)
                        in_trade_long = True
                        trade_num += 1
    
                    if in_trade_long:
                        if close_price <= low_level:
                            # SL Hit
                            # print('SL Condition PE')
                            for trade in tradebook:
                                if (
                                    trade['type'] == 'PE' and 
                                    trade['date'] == current_date and 
                                    'exit_time' not in trade
                                ):
                                    df_row = pe_df_pandas[pe_df_pandas['datetime'] <= timestamp]
                                    # print(df_row.to_string())
                                    pe_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                                    trade['exit price'] =pe_exit_price
                                    trade['exit_time'] = df_row['datetime'].iloc[-1] if len(df_row) != 0 else float('nan')
                                    trade['remarks'] = 'SL Hit'
                                    
                            in_trade_long = False
                            low_flag_2 = False
                            
                        elif timestamp.time() >= dt.time(15, 20):
                            #EOD Exit
                            # print('EOD Exit Condition PE')
                            for trade in tradebook:
                                if (
                                    trade['type'] == 'PE' and 
                                    trade['date'] == current_date and 
                                    'exit_time' not in trade
                                ):
                                    df_row = pe_df_pandas[pe_df_pandas['datetime'] <= timestamp]
                                    # print("EOD PE Side")
                                    # print(df_row.to_string())
                                    pe_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                                    trade['exit price'] =pe_exit_price
                                    trade['exit_time'] = df_row['datetime'].iloc[-1] if len(df_row) != 0 else float('nan')
                                    trade['remarks'] = 'EOD Exit'
                                    
                            in_trade_long = False
                            low_flag_2 = False
        
        current_date += dt.timedelta(days=1)
    tb = pd.DataFrame(tradebook)
    if len(tb) > 0:
        tb['points'] = tb['entry price'] - tb['exit price']
        tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
        tb['final_points'] = tb['points'] - tb['slippage']
        tb['portfolio'] = PORTFOLIO_
        tb['index_lev'] = INDEX_LEVERAGE_
        tb['qty'] = tb['portfolio'] * tb['index_lev'] / tb['strike']
        tb['pnl'] = tb['final_points'] * tb['qty']
        tb['ROI%'] = tb['pnl'] * 100 / tb['portfolio']
    return tb

In [77]:
TF_ = '3m'
OFFSET_ = '0m'
MULTIPLIER_ = 3
ATR_WINDOW_ = 14
open_range_till = dt.time(9, 20)
bnf_df = resample(pl.DataFrame(bnf_1min), TF_, OFFSET_)
bnf_df_pandas = bnf_df.to_pandas()
bnf_df_pandas = await add_atr(bnf_df_pandas, ATR_WINDOW_)
tb = await execute(bnf_df_pandas, MULTIPLIER_, open_range_till, TF_, OFFSET_)

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-18
2019-03-19
2019-03-20
2019-03-22
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-04-01
2019-04-02
2019-04-03
2019-04-04
2019-04-05
2019-04-08
2019-04-09
2019-04-10
2019-04-11
2019-04-12
2019-04-15
2019-04-16
2019-04-18
2019-04-22
2019-04-23
2019-04-24
2019-04-25
2019-04-26
2019-04-30
2019-05-02
2019-05-03
2019-05-06
2019-05-07
2019-05-08
2019-05-09
2019-05-10
2019-05-13
2019-05-14
2019-05-15

In [78]:
tb

,date,high level,low level,atr_multiplier,side,strike,type,expiry,entry_time,entry price,exit price,exit_time,remarks,points,slippage,final_points,portfolio,index_lev,qty,pnl,ROI%
0,2019-01-01,10912.8000,10840.5000,3,LONG,10900,PE,2019-01-31,2019-01-01 09:27:00,195.0000,202.0000,2019-01-01 09:33:00,SL Hit,-7.0000,3.9700,-10.9700,10000000,8,7339.4495,-80513.7615,-0.8051
1,2019-01-01,10912.8000,10840.5000,3,LONG,10900,PE,2019-01-31,2019-01-01 09:36:00,198.0000,198.1000,2019-01-01 09:51:00,SL Hit,-0.1000,3.9610,-4.0610,10000000,8,7339.4495,-29805.5046,-0.2981
2,2019-01-03,10809.5842,10736.6158,3,SHORT,10750,CE,2019-01-31,2019-01-03 10:21:00,238.0000,166.4500,2019-01-03 15:21:00,EOD Exit,71.5500,4.0445,67.5055,10000000,8,7441.8605,502366.5116,5.0237
3,2019-01-04,10740.3166,10669.9834,3,LONG,10750,PE,2019-01-31,2019-01-04 09:51:00,196.2000,198.4000,2019-01-04 09:54:00,SL Hit,-2.2000,3.9460,-6.1460,10000000,8,7441.8605,-45737.6744,-0.4574
4,2019-01-04,10740.3166,10669.9834,3,LONG,10750,PE,2019-01-31,2019-01-04 11:00:00,190.0000,161.5000,2019-01-04 15:21:00,EOD Exit,28.5000,3.5150,24.9850,10000000,8,7441.8605,185934.8837,1.8593
5,2019-01-07,10862.3460,10769.0540,3,LONG,10850,PE,2019-01-31,2019-01-07 14:54:00,190.9500,199.6000,2019-01-07 14:57:00,SL Hit,-8.6500,3.9055,-12.5555,10000000,8,7373.2719,-92575.1152,-0.9258
6,2019-01-08,10770.3811,10705.8189,3,SHORT,10700,CE,2019-01-31,2019-01-08 10:48:00,228.5000,232.3000,2019-01-08 10:51:00,SL Hit,-3.8000,4.6080,-8.4080,10000000,8,7476.6355,-62863.5514,-0.6286
7,2019-01-08,10770.3811,10705.8189,3,SHORT,10700,CE,2019-01-31,2019-01-08 10:54:00,222.7000,235.8500,2019-01-08 12:45:00,SL Hit,-13.1500,4.5855,-17.7355,10000000,8,7476.6355,-132601.8692,-1.3260
8,2019-01-09,10904.0153,10830.3847,3,LONG,10900,PE,2019-01-31,2019-01-09 12:36:00,167.6000,183.4000,2019-01-09 13:51:00,SL Hit,-15.8000,3.5100,-19.3100,10000000,8,7339.4495,-141724.7706,-1.4172
9,2019-01-09,10904.0153,10830.3847,3,LONG,10900,PE,2019-01-31,2019-01-09 14:51:00,159.9000,158.4500,2019-01-09 15:21:00,EOD Exit,1.4500,3.1835,-1.7335,10000000,8,7339.4495,-12722.9358,-0.1272


In [83]:
tb.to_csv('Check-JJMS-ATR-Intraday.csv', index=False)

In [80]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{variation}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [81]:
tb['entry_time'] = pd.to_datetime(tb['entry_time'])
tb["Trade Year"] = tb["entry_time"].dt.year
stats = generate_stats(tb, 'BASE Variation')

In [82]:
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,-33.5752,354,14.1243,2.0734,-0.4652,-38.8977,-0.8632,BASE Variation
2020,-86.5257,338,12.7219,2.9420,-0.7423,-96.0603,-0.9007,BASE Variation
2021,-44.6268,332,17.4699,1.7233,-0.5277,-44.9187,-0.9935,BASE Variation
2022,-92.3128,334,11.0778,2.4605,-0.6236,-96.7089,-0.9545,BASE Variation
2023,-7.9856,329,19.1489,1.5790,-0.4165,-15.2818,-0.5226,BASE Variation
2024,-53.4976,310,17.7419,1.7584,-0.5891,-59.3182,-0.9019,BASE Variation
2025,-15.9600,91,16.4835,1.6753,-0.5407,-25.2884,-0.6311,BASE Variation
Overall,-334.4836,2088,15.3736,2.0015,-0.5618,-339.8060,-0.9843,BASE Variation
